#### Healthier Comparison McDonald's vs _Starbucks_

In [0]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col
from pyspark.sql.functions import lit
from pyspark.sql.functions import avg

In [0]:
spark = SparkSession.builder \
    .appName("Healthier_Comparison") \
    .getOrCreate()

#### 1) Convert all xlsx file to CSV
#### 2) Upload files in DBFS

In [0]:
%fs ls /FileStore/tables/

path,name,size,modificationTime
dbfs:/FileStore/tables/Sample_Data.csv,Sample_Data.csv,104,1731676084000
dbfs:/FileStore/tables/mealDatabase.csv,mealDatabase.csv,37424,1732114017000
dbfs:/FileStore/tables/menu.csv,menu.csv,30233,1740139024000
dbfs:/FileStore/tables/restaurant.json,restaurant.json,681632,1731670025000
dbfs:/FileStore/tables/starbucks_drinkMenu_expanded.csv,starbucks_drinkMenu_expanded.csv,26907,1740139024000
dbfs:/FileStore/tables/starbucks_menu_nutrition_drinks.csv,starbucks_menu_nutrition_drinks.csv,8531,1740139025000
dbfs:/FileStore/tables/starbucks_menu_nutrition_food.csv,starbucks_menu_nutrition_food.csv,5038,1740139025000
dbfs:/FileStore/tables/tips.csv,tips.csv,8188,1731591548000
dbfs:/FileStore/tables/userdata.parquet,userdata.parquet,113629,1731672145000


- dbfs:/FileStore/tables/menu.csv
- dbfs:/FileStore/tables/starbucks_drinkMenu_expanded.csv
- dbfs:/FileStore/tables/starbucks_menu_nutrition_drinks.csv
- dbfs:/FileStore/tables/starbucks_menu_nutrition_food.csv

- md_df = menu.csv
- sb_drink_df = starbucks_drinkMenu_expanded.csv
- sb_nutD_df = starbucks_menu_nutrition_drinks.csv
- sb_nutF_df = starbucks_menu_nutrition_food.csv

In [0]:
# menu.csv
mcd_df = spark.read.format("csv") \
    .option("header","true") \
    .option("inferSchema","true") \
    .load("dbfs:/FileStore/tables/menu.csv")

# starbucks_drinkMenu_expanded.csv
sb_drink_df = spark.read.format("csv") \
    .option("header","true") \
    .option("inferSchema","true") \
    .load("dbfs:/FileStore/tables/starbucks_drinkMenu_expanded.csv")

# starbucks_menu_nutrition_drinks.csv
sb_nutD_df = spark.read.format("csv") \
    .option("header","true") \
    .option("inferSchema","true") \
    .load("dbfs:/FileStore/tables/starbucks_menu_nutrition_drinks.csv")
    
# starbucks_menu_nutrition_food.csv
sb_nutF_df = spark.read.format("csv") \
    .option("header","true") \
    .option("inferSchema","true") \
    .load("dbfs:/FileStore/tables/starbucks_menu_nutrition_food.csv")

### Cleaning and selecting IMP columns

##### Selecting Columns for McDonald's

In [0]:
mcd_df = mcd_df.select(
    col("Item").alias("Food_Item"),
    col("Calories").cast("int"),
    col("Total Fat").alias("Fat").cast("float"),
    col("Sugars").cast("float"),
    col("Protein").cast("float"),
    col("Carbohydrates").alias("Carbs").cast("float")
).dropna()

##### Add new column Brand with litrial McDonald's

In [0]:
mcd_df = mcd_df.withColumn("Brand", lit("McDonald's"))
mcd_df.display()

Food_Item,Calories,Fat,Sugars,Protein,Carbs,Brand
Egg McMuffin,300,13.0,3.0,17.0,31.0,McDonald's
Egg White Delight,250,8.0,3.0,18.0,30.0,McDonald's
Sausage McMuffin,370,23.0,2.0,14.0,29.0,McDonald's
Sausage McMuffin with Egg,450,28.0,2.0,21.0,30.0,McDonald's
Sausage McMuffin with Egg Whites,400,23.0,2.0,21.0,30.0,McDonald's
Steak & Egg McMuffin,430,23.0,3.0,26.0,31.0,McDonald's
"Bacon, Egg & Cheese Biscuit (Regular Biscuit)",460,26.0,3.0,19.0,38.0,McDonald's
"Bacon, Egg & Cheese Biscuit (Large Biscuit)",520,30.0,4.0,19.0,43.0,McDonald's
"Bacon, Egg & Cheese Biscuit with Egg Whites (Regular Biscuit)",410,20.0,3.0,20.0,36.0,McDonald's
"Bacon, Egg & Cheese Biscuit with Egg Whites (Large Biscuit)",470,25.0,4.0,20.0,42.0,McDonald's


##### Selecting Columns for StarBucks

##### Select from starbucks_menu_nutrition_food

In [0]:
sb_nutF_df = sb_nutF_df.select(
    col("_c0").alias("Food_Item"),  # If the first column is unnamed, use _c0
    col("Calories").cast("int"),
    col("Fat (g)").alias("Fat").cast("float"),
    col("`Carb. (g)`").alias("Carbs").cast("float"), # Use backticks (``) around column names if they contain spaces or dots
    col("Protein (g)").alias("Protein").cast("float")
).dropna()

##### Select from starbucks_menu_nutrition_drinks

In [0]:
sb_nutD_df = sb_nutD_df.select(
    col("_c0").alias("Food_Item"),
    col("Calories").cast("int"),
    col("Fat (g)").alias("Fat").cast("float"),
    col("`Carb. (g)`").alias("Carbs").cast("float"),
    col("Protein").cast("float")
).dropna()

#### combining both sb_nutD_df & sb_nutF_df 
#### and Add new column Brand with data Starbucks

In [0]:
sb_df = sb_nutF_df.union(sb_nutD_df).withColumn("Brand", lit("Starbucks"))

In [0]:
sb_df = sb_df.withColumn("Sugars", lit(None).cast("float"))

In [0]:
mcd_df.display()

Food_Item,Calories,Fat,Sugars,Protein,Carbs,Brand
Egg McMuffin,300,13.0,3.0,17.0,31.0,McDonald's
Egg White Delight,250,8.0,3.0,18.0,30.0,McDonald's
Sausage McMuffin,370,23.0,2.0,14.0,29.0,McDonald's
Sausage McMuffin with Egg,450,28.0,2.0,21.0,30.0,McDonald's
Sausage McMuffin with Egg Whites,400,23.0,2.0,21.0,30.0,McDonald's
Steak & Egg McMuffin,430,23.0,3.0,26.0,31.0,McDonald's
"Bacon, Egg & Cheese Biscuit (Regular Biscuit)",460,26.0,3.0,19.0,38.0,McDonald's
"Bacon, Egg & Cheese Biscuit (Large Biscuit)",520,30.0,4.0,19.0,43.0,McDonald's
"Bacon, Egg & Cheese Biscuit with Egg Whites (Regular Biscuit)",410,20.0,3.0,20.0,36.0,McDonald's
"Bacon, Egg & Cheese Biscuit with Egg Whites (Large Biscuit)",470,25.0,4.0,20.0,42.0,McDonald's


In [0]:
sb_df.display()

Food_Item,Calories,Fat,Sugars,Protein,Carbs,Brand
Chonga Bagel,300,5.0,null,12.0,50.0,Starbucks
8-Grain Roll,380,6.0,null,10.0,70.0,Starbucks
Almond Croissant,410,22.0,null,10.0,45.0,Starbucks
Apple Fritter,460,23.0,null,7.0,56.0,Starbucks
Banana Nut Bread,420,22.0,null,6.0,52.0,Starbucks
Blueberry Muffin with Yogurt and Honey,380,16.0,null,6.0,53.0,Starbucks
Blueberry Scone,420,17.0,null,5.0,61.0,Starbucks
Butter Croissant,240,12.0,null,5.0,28.0,Starbucks
Butterfly Cookie,350,22.0,null,2.0,38.0,Starbucks
Cheese Danish,320,16.0,null,8.0,36.0,Starbucks


##### sequencing column of sb_df to union with mcd_df

In [0]:
sb_df = sb_df.select("Food_Item", "Calories", "Fat", "Sugars", "Protein", "Carbs", "Brand")

##### combining both mcd_df & sb_df

In [0]:
combined_df = mcd_df.union(sb_df)

In [0]:
combined_df.display()

Food_Item,Calories,Fat,Sugars,Protein,Carbs,Brand
Egg McMuffin,300,13.0,3.0,17.0,31.0,McDonald's
Egg White Delight,250,8.0,3.0,18.0,30.0,McDonald's
Sausage McMuffin,370,23.0,2.0,14.0,29.0,McDonald's
Sausage McMuffin with Egg,450,28.0,2.0,21.0,30.0,McDonald's
Sausage McMuffin with Egg Whites,400,23.0,2.0,21.0,30.0,McDonald's
Steak & Egg McMuffin,430,23.0,3.0,26.0,31.0,McDonald's
"Bacon, Egg & Cheese Biscuit (Regular Biscuit)",460,26.0,3.0,19.0,38.0,McDonald's
"Bacon, Egg & Cheese Biscuit (Large Biscuit)",520,30.0,4.0,19.0,43.0,McDonald's
"Bacon, Egg & Cheese Biscuit with Egg Whites (Regular Biscuit)",410,20.0,3.0,20.0,36.0,McDonald's
"Bacon, Egg & Cheese Biscuit with Egg Whites (Large Biscuit)",470,25.0,4.0,20.0,42.0,McDonald's


##### Calculating avg of calories, fat, sugar, carbs, protein

In [0]:
summary_df = combined_df.groupBy("Brand").agg(
    avg("Calories").alias("Avg_Calories"),
    avg("Fat").alias("Avg_Fat"),
    avg("Sugars").alias("Avg_Sugars"),
    avg("Carbs").alias("Avg_Carbs"),
    avg("Protein").alias("Avg_Protein")
)

### Result

In [0]:
summary_df.show()

+----------+-----------------+------------------+------------------+-----------------+------------------+
|     Brand|     Avg_Calories|           Avg_Fat|        Avg_Sugars|        Avg_Carbs|       Avg_Protein|
+----------+-----------------+------------------+------------------+-----------------+------------------+
|McDonald's|368.2692307692308|14.165384615384616|29.423076923076923|47.34615384615385|13.338461538461539|
| Starbucks|257.2439024390244| 10.06341463414634|              null|33.97073170731707| 8.185365853658537|
+----------+-----------------+------------------+------------------+-----------------+------------------+

